### 1. 2020, 2021 데이터 병합
### 2.  사건종별 칼럼 중, 교통 사고 관련 인덱스만 불러오기
### 3. 각 칼럼별 unique() 확인

In [1]:
import pandas as pd
import numpy as np

C:\Users\user\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [138]:
df1 = pd.read_csv("D:/치안빅데이터/데이터/KP2020.csv", encoding='euc-kr')
df2 = pd.read_csv("D:/치안빅데이터/데이터/KP2021.csv", encoding='euc-kr')


def code(df1,df2):

    df3 = pd.concat([df1,df2], axis=0)
    df3 = df3.reset_index(drop=True)

    col = ['접수부서', '접수완료일시', '경찰청구분', '사건상태', '사건종별',
           '신고성별', '발생지점', '발생좌표x', '발생좌표y', '동일사건여부']

    df3.columns = col
    
    tram = df3[df3['사건종별'].isin([401, 402, 403, 404, 405,406])]
    tram = tram.reset_index(drop=True)

    return(tram)

tram = code(df1,df2)

In [15]:
tram.shape

(287669, 10)

In [139]:
tram.isnull().sum()

접수부서           0
접수완료일시         0
경찰청구분          0
사건상태           0
사건종별           0
신고성별           0
발생지점        7478
발생좌표x       1866
발생좌표y       1866
동일사건여부    218799
dtype: int64

In [16]:
for i in tram.columns:
    print('칼럼명:',i)
    print('-'*100)
    
    print(tram[i].unique())
    print('-'*100)

칼럼명: 접수부서
----------------------------------------------------------------------------------------------------
['충남청' '대전청' '2지구대(충남)' '대전파출소' '대전중부' '대전유성' '서대전지구대' '남대전지구대' '대전역지구대'
 '세종' '대전동부' '세종청' '대전둔산' '대전대덕' '세종로파출소' '대전서부' '세종서' '대전동부서' '대전서부서'
 '대전중부서' '대전둔산서' '대전유성서' '대전대덕서' '세종남부서' '세종북부서' '세종대왕파출소']
----------------------------------------------------------------------------------------------------
칼럼명: 접수완료일시
----------------------------------------------------------------------------------------------------
['20/12/01 08:17:50.000000000' '20/12/01 03:24:26.000000000'
 '20/12/01 07:37:18.000000000' ... '23/01/14 01:02:48.000000000'
 '23/01/14 17:35:38.000000000' '23/01/15 01:13:20.000000000']
----------------------------------------------------------------------------------------------------
칼럼명: 경찰청구분
----------------------------------------------------------------------------------------------------
[19 13 31 21 11 18 15 20 10 16  8 17 22 23 12 14  9 24]
--------------

### 4. 필요 컬럼 생성
#### (1) 접수완요일시 세분화 (연,월,일, 시간)
#### (2) 접수지역 세분화 (발생시, 동, 구)

#### (1) 접수완료일시 세분화

In [140]:
def time_split(tram):
    
    # 접수 연,월,일, 시간 세분화
    tram['접수완료날짜'] = tram['접수완료일시'].str.split(' ').str[0]
    tram['접수완료시간대'] = tram['접수완료일시'].str.split(' ').str[1]
    
    tram['접수연도'] = tram['접수완료날짜'].str.split('/').str[0]
    tram['접수월'] = tram['접수완료날짜'].str.split('/').str[1]
    tram['접수일자'] = tram['접수완료날짜'].str.split('/').str[2]
    
    tram['접수완료시간'] = tram['접수완료시간대'].str.split(':').str[0]
    tram['접수완료시간대'] = tram['접수완료시간대'].str.split('.').str[0]
    tram = tram.drop(columns='접수완료일시',axis=1)
    
    return(tram)

In [141]:
tram = time_split(tram)
tram.head()

,접수부서,경찰청구분,사건상태,사건종별,신고성별,발생지점,발생좌표x,발생좌표y,동일사건여부,접수완료날짜,접수완료시간대,접수연도,접수월,접수일자,접수완료시간
0,충남청,19,10,401,2.0,충청남도 천안시 서북구 성정동(행정:성정2동) 1259,127.137160,36.826718,NaN,20/12/01,08:17:50,20,12,01,08
1,대전청,13,10,406,1.0,대전광역시 유성구 봉명동(행정:온천1동) 575,127.341551,36.353696,NaN,20/12/01,03:24:26,20,12,01,03
2,충남청,19,10,402,1.0,충청남도 천안시 서북구 성환읍 성월리(행정:성환읍) 150,127.136142,36.917900,NaN,20/12/01,07:37:18,20,12,01,07
3,대전청,13,10,401,1.0,대전광역시 중구 석교동(석교동) 85-14,127.447545,36.306724,NaN,20/12/01,07:08:44,20,12,01,07
4,대전청,13,10,401,2.0,대전광역시 중구 석교동(행정:석교동) 88-11,127.447175,36.306694,NaN,20/12/01,07:09:20,20,12,01,07


#### 지역세분화 형식
#### (1) 광역시/도       (2) 시군구             (3) 읍면동
#### 열  대전,충남,세종      유성구, 천안시         온천동, 불당동, 조치원읍
#### 이런 형식

In [47]:
c = tram['발생지점'].str.split(' ').str[0] == '충청남도'
b = tram['발생지점'].str.split(' ').str[0] == '대전광역시'
a = tram['발생지점'].str.split(' ').str[0] == '세종특별자치시'

tram[a].shape[0] + tram[b].shape[0] + tram[c].shape[0] 

271608

#### 충청남도, 대전광역시, 세종특별자치시를 합쳤을때 열이 27만개인데 기존 열 개수가 약 28만개로 약 1만개의 잘못된 데이터 존재!
#### 가령 발생지점이 충남지역이 아니라던가, 충청남도, 대전광역시가 아닌 충남, 대전으로 표기된 경우!

#### 방법1: 일단 먼저 split해서 광역시/도, 시군구, 읍면동 칼럼 생성 후, 특이한 이름들이 왜나왔는지 살펴본다
#### 방법2: 데이터가 약 27만개이므로 광역시/도가 호서 지역 관련이 아니면서 데이터가 100개 미만인 데이터는 지운다 

In [96]:
k = tram['발생지점'].str.split(' ').str[0]

do = k.value_counts()
do_df = pd.DataFrame({'빈도':do})
do_df.head(20).T

,충청남도,대전광역시,세종특별자치시,대전,경기도,충청북도,충남,전라북도,전라남도,서울특별시,경상북도,인천광역시,대구광역시,강원도,경상남도,부산광역시,광주광역시,제주특별자치도,[공중전화],[이관주소]
빈도,155424,99154,17030,4259,1376,1156,570,471,157,135,60,52,40,31,26,25,21,18,18,18


In [101]:
k1 = tram['발생지점'].str.split(' ').str[1]

do1 = k1.value_counts()
do1_df = pd.DataFrame({'빈도':do1})
do1_df[do1_df['빈도'] > 100].T

,천안시,서구,아산시,유성구,세종시,중구,동구,대덕구,당진시,서산시,...,부여군,계룡시,청양군,청주시,평택시,안성시,군산시,익산시,옥천군,담양군
빈도,52926,33516,23810,23586,16834,16145,15563,14658,13938,12049,...,2644,1364,1261,817,661,288,158,152,148,115


In [142]:
tram['광역시/도'] = tram['발생지점'].str.split(' ').str[0]
tram['시군구'] = tram['발생지점'].str.split(' ').str[1]
tram['읍면동'] = tram['발생지점'].str.split(' ').str[2]

#### 방법1 적용
#### [이관주소], [공중전화]가 있는 부분은 ex) [이관주소] 충청남도 천안시 ~~ 형식이므로 다르게 처리해줘야함!
#### [이관주소], [공중전화]라는 단어를 공백으로 replace 한다!

In [144]:
tram['광역시/도'] = tram['광역시/도'].replace('[공중전화]','')
tram['광역시/도'] = tram['광역시/도'].replace('[이관주소]','')

#### 방법2 적용 
#### 발생지점의 띄어쓰기 기준 0번째 단어에 우리가 생각한건 충청남도, 대전광역시, 세종특별자치시인데 다른지역+이상한 글자도 있음
#### 기본적으로 발생지점이 충남, 대전 지역이 아닌 곳은 다 지운다!

In [145]:
# 호서 지역의 광역시/도만 불러옴 
a1 = tram['광역시/도'] == '충청남도'
a2 = tram['광역시/도'] == '대전광역시'
a3 = tram['광역시/도'] == '세종특별자치시'
a4 = tram['광역시/도'] == '대전'
a5 = tram['광역시/도'] == '충남'

tram = tram[a1|a2|a3|a4|a5]

# 대전과 충남의 이름을 통일 
tram['광역시/도'] = tram['광역시/도'].replace('대전','대전광역시')
tram['광역시/도'] = tram['광역시/도'].replace('충남','충청남도')

In [129]:
tram['광역시/도'].value_counts()

충청남도       155994
대전광역시      103413
세종특별자치시     17030
Name: 광역시/도, dtype: int64

In [134]:
sejong = tram[tram['광역시/도'] == '세종특별자치시']
sejong[sejong['시군구'] == '세종특별자치시'].head(3)

,접수부서,경찰청구분,사건상태,사건종별,신고성별,발생지점,발생좌표x,발생좌표y,동일사건여부,접수완료날짜,접수완료시간대,접수연도,접수월,접수일자,접수완료시간,광역시/도,시군구,읍면동
51917,충남청,31,10,401,1.0,세종특별자치시 세종특별자치시 다정동(새롬동) 22-8,127.243519,36.492812,NaN,21/05/14,20:05:43,21,05,14,20,세종특별자치시,세종특별자치시,다정동(새롬동)
63091,충남청,31,10,401,1.0,세종특별자치시 세종특별자치시 다정동(새롬동) 321-4,127.242981,36.495979,Y,21/06/16,23:14:55,21,06,16,23,세종특별자치시,세종특별자치시,다정동(새롬동)
68913,충남청,31,10,406,3.0,세종특별자치시 세종특별자치시 다정동(새롬동) 22-8,127.243519,36.492812,NaN,21/06/28,23:00:17,21,06,28,23,세종특별자치시,세종특별자치시,다정동(새롬동)


#### 시군구 칼럼에도 '세종시'가 아니라 세종특별자치시 존재 세종시로 바꿔준다!

In [147]:
tram['시군구'] = tram['시군구'].replace('세종특별자치시','세종시')

In [185]:
no_see = tram[tram['시군구'] =='']
no_see['광역시/도'].value_counts()

세종특별자치시    88
Name: 광역시/도, dtype: int64

#### 시군구 칼럼에 아무것동 없는 데이터 88개 존재
#### 살펴보니 세종특별자치시 ㅇㅇ동으로 되어있는 열들임 -> 시군구에 세종시를 넣어주자!

In [186]:
tram['시군구'] = tram['시군구'].replace('','세종시')

### 5. 데이터 타입 변경

In [148]:
tram = tram.astype({'접수연도':'int'})
tram = tram.astype({'접수일자':'int'})
tram = tram.astype({'접수월':'int'})
tram= tram.astype({'접수완료시간':'int'})

### 결측치 

In [149]:
tram.isnull().sum()

접수부서            0
경찰청구분           0
사건상태            0
사건종별            0
신고성별            0
발생지점            0
발생좌표x          28
발생좌표y          28
동일사건여부     209799
접수완료날짜          0
접수완료시간대         0
접수연도            0
접수월             0
접수일자            0
접수완료시간          0
광역시/도           0
시군구             0
읍면동             0
dtype: int64

#### 1. 동일사건여부 컬럼은 보간 불가하다 판단하여 제거 

In [150]:
tram = tram.drop(columns=['동일사건여부'],axis=1)

#### 2. 발생좌표가 없는 열 보간

In [153]:
x = tram['발생좌표x'].isnull()
y = tram['발생좌표y'].isnull()
no = tram[x&y]

In [159]:
address = no['발생지점']
print(address.head())

369           충청남도 천안시 서북구 두정동(부성2동) 955
416       충청남도 천안시 서북구 불당동(행정:불당동) 234-1
523       충청남도 태안군 태안읍 남문리(행정:태안읍) 720-9
601     충청남도 천안시 동남구 목천읍 신계리(행정:목천읍) 422
1045          충청남도 공주시 신관동(행정:신관동) 24-51
Name: 발생지점, dtype: object


In [156]:
pip install geopy

     -------------------------------------- 119.8/119.8 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00



In [161]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [162]:
발생좌표x = []
발생좌표y =[]

for i in address:
    발생좌표x.append(geocoding(i)[0])
    발생좌표y.append(geocoding(i)[1])

In [167]:
print(발생좌표x)
print(발생좌표y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36.8239571, 36.8024722, 36.8119683, 36.8146887, 0, 36.7840779, 0, 0, 36.4328828, 0, 0, 0, 36.60379555, 0, 0, 36.3298999, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 127.14010219276282, 127.12952356570844, 127.1089675, 127.1808139, 0, 127.1253344, 0, 0, 127.2777721, 0, 0, 0, 127.29381782866696, 0, 0, 127.4066297, 0]


#### 특정 주소는 불러오지만 대부분 못불러옴 -> 엑셀로 받아서 지오코딩할 필요성 존재 !

In [188]:
tram.to_csv('전처리95퍼완.csv',encoding='euc-kr')